Dataset:

Het is een audiodataset. Een audio dataset heeft 3 dimensies. In pytorch is dat de 1d dimensie.

10 klassen:
13 features

8800 observaties
10 getallen
10 herhalingen
88 sprekers (88x10x10)

1 regel getallen is 1 uitspraak van 1 persoon
1 blok is 1 persoon die 1 uitspraak doet (getal 3 bijv)

Potentieel probleem: Te weinig observaties? Te weinig sprekers? Relatief kleine dataset.

Hoe bepaal je de grootte van de filters?
- Relatief kleine filters gebruiken omdat de datasets maar 13 features hebben.
- input channels: Gelijk aan het aantal features. In dit geval 13.
- In een linear model de hidden state niet te ver oprekken, kans op overfitting




Modellen om te proberen:
CNN
Gru 
Gru + attention



In [ ]:
#Attention GRU template

class AttentionGRU(nn.Module):
    def __init__(
        self,
        config: Dict,
    ) -> None:
        super().__init__()
        self.rnn = nn.GRU(
            input_size=config["input_size"],
            hidden_size=config["hidden_size"],
            dropout=config["dropout"],
            batch_first=True,
            num_layers=config["num_layers"],
        )
        self.attention = nn.MultiheadAttention(
            embed_dim=config["hidden_size"],
            num_heads=4,
            dropout=config["dropout"],
            batch_first=True,
        )
        self.linear = nn.Linear(config["hidden_size"], config["output_size"])

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        x, _ = self.attention(x.clone(), x.clone(), x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat